<a href="https://colab.research.google.com/github/tgialoimtr/backbone/blob/master/dental_seqconv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import os, sys
import json
import tensorflow as tf

In [ ]:
!ls "/content/gdrive/My Drive/workspace/dental/"

checkpoints  checkpoints2  tb  tb2


In [ ]:
!nvidia-smi

Wed Aug 19 09:18:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 1244x(2245~2976) => 1x1.8~2.3tf.__version__
# 1350x(3118~3407) => 1x2.3~2.5

In [ ]:
import hashlib
 
def str2col(s):
    ''' convert/hash string to color, so that it describe same string. '''
    RGBint = int(hashlib.sha1(s.encode('utf-8')).hexdigest(), 16) % (2 ** 24)
    blue =  RGBint & 255
    green = (RGBint >> 8) & 255
    red =   (RGBint >> 16) & 255
    return red, green, blue
 
def name2desc(name):
    ''' 
    FDI labeling rule: example 14x
    1 means upper-left
    4 means 4th teeth
    x: optional, in [m, g, i, e]
    -m: missing root of teeth (only top)
    -g: only root exist, missing top
    -i: implant
    -e: extended teeth (bij duw rawng)
    '''
    rs = {'top':False, 'left':False, 'major':0, 'minor':0, 'ext':None}
    rs['major'] = int(name[0])
    rs['minor'] = int(name[1])
    if name[0] == '1' or name[0] == '2':
        rs['top'] = True
    if name[0] == '3' or name[0] == '2':
        rs['left'] = True
    if len(name) > 2:
        rs['ext'] = name[2]
    return rs
 
class Contour(object):
    
    def __init__(self, name, points):
        self.name = name
        self.desc = name2desc(name)
        self.xy_points = np.array(points) # (x,y) points
        self.points = np.stack([self.xy_points[:,1], self.xy_points[:,0]], axis=1) # (y,x) points
        self.center = np.mean(self.points, axis=0)
        
    def lowest_point(self):
        lowest_point_idx = np.argmax(self.points[:,0])
        return self.points[lowest_point_idx, :]
    
    def highest_point(self):
        highest_point_idx = np.argmin(self.points[:,0])
        return self.points[highest_point_idx, :]
        
    def draw(self, img, col_str=None, what_to_draw=['contour', 'label']):
        xyi_points = self.xy_points.astype(np.int32)
        if col_str is None:
            col = str2col(self.name)
        else:
            col = str2col(col_str)
        if 'contour' in what_to_draw:
            cv2.drawContours(img, [xyi_points], -1, col, 2)
        if 'fill' in what_to_draw:
            cv2.drawContours(img, [xyi_points], -1, col, -1)
        if 'label' in what_to_draw:
            if 'fill' in what_to_draw: col = (255-col[0], 255 - col[1], 255 - col[2])
            pos = (int(self.center[1]), int(self.center[0]))
            cv2.putText(img, self.name, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, col)

In [ ]:
ext_masks = ['i', 'm', 'e', 'b', 'g']
ext_masks.index('m')
 
ext_labels = ['i', 'm', 'e', 'b', 'g']
teeth_labels = ['11','12','13','14','15','16','17','18',\
              '21','22','23','24','25','26','27','28',\
              '31','32','33','34','35','36','37','38',\
              '41','42','43','44','45','46','47','48']

In [ ]:
def gen(img, lbl):
    allpoints = np.concatenate([np.array(shape['points']) for shape in lbl['shapes']])
    xmin, ymin = allpoints.min(axis=0)
    xmax, ymax = allpoints.max(axis=0)
    max_pad_x = (xmax - xmin)*0.07
    max_pad_y = (ymax - ymin)*0.07
    import random
    xmax += random.uniform(0, max_pad_x)
    xmin -= random.uniform(0, max_pad_x)
    ymax += random.uniform(0, max_pad_y)
    ymin -= random.uniform(0, max_pad_y)
 
    img_croped = img[int(ymin):int(ymax), int(xmin):int(xmax), :]
    img_croped = cv2.resize(img_croped, (512,256))
 
    ry = 256.0/(ymax-ymin)
    rx = 512.0/(xmax-xmin)
 
    masks = np.zeros(shape=(len(teeth_labels) + len(ext_labels),256,512), dtype=np.uint8)
    m = np.zeros(shape=(256,512), dtype=np.uint8)
 
    for shape in lbl['shapes']:
        c = Contour(shape['label'], shape['points'])
        i = teeth_labels.index(c.name[:2])
    #     mask = np.zeros(shape=(h,w))
        pts = c.xy_points
        pts -= [xmin,ymin]
        pts = (pts*[rx,ry]).astype(int)
        #print(pts)
        cv2.drawContours(masks[i], [pts], -1, 255, -1)
        m = cv2.add(m, masks[i]//255)
        if len(c.name) > 2:
            j = ext_labels.index(c.name[2])
            masks[len(teeth_labels) + j] = cv2.bitwise_or(masks[len(teeth_labels) + j], masks[i])
 
    labels_masks = np.rollaxis(masks, 0, 3)
    premask = np.stack([(m==0), (m==1), (m>1)], axis=2)
    return img_croped, premask, labels_masks > 0

In [ ]:
# img_dir = '/home/loitg/ssd/data/dental_data/images/2007 387枚 読影済/'
# lbl_dir = '/home/loitg/ssd/data/dental_data/json/2007 387枚 読影済/'
# temp_dir = '/home/loitg/workspace/dental/temp'
 
# fn = '20070508105519'
# imgfn = fn + '.png'
# lblfn = fn + '.json'
# img = cv2.imread(os.path.join(img_dir, imgfn))
 
# lbl = json.load(open(os.path.join(lbl_dir, lblfn)))
 
# img_croped, premask, labels_masks = gen(img, lbl)

In [ ]:
# np.concatenate([labels_masks, premask], axis=2).shape

In [ ]:
from glob import glob
import random
 
temp_dir = '/content/gdrive/My Drive/workspace/dental/temp/'
root_data = '/content/gdrive/My Drive/workspace/dental/'
all_json = glob('/content/gdrive/My Drive/workspace/dental_data/json/*/*.json')
 
def kkk():
    random.shuffle(all_json)
    for path_json in all_json:
        try:
            path_png = path_json.replace('.json','.png').replace('json','images')
            img = cv2.imread(path_png)
            if len(img.shape) < 3 or img.shape[2] < 3:
                img = cv2.ctvColor(img, cv2.COLOR_GRAY2BGR)
            lbl = json.load(open(path_json))
            _, fn = os.path.split(path_png)
            # print(fn)
            img_croped, premask, labels_masks = gen(img, lbl)
            img_normalized = img_croped/128.0 - 1
            yield img_normalized, (premask, labels_masks)
        except Exception as e:
            print('Exception!!!')
            continue

In [ ]:
import tensorflow as tf
train_ds = tf.data.Dataset.from_generator(kkk, args=[], output_types=(tf.float64,(tf.uint8,tf.uint8)) ,\
                                    output_shapes = ((256, 512, 3), ((256, 512, 3), (256, 512, 37)) ))
train_ds = train_ds.batch(4)

In [ ]:
val_ds = train_ds.take(4)
val_ds = list(val_ds)

Exception!!!


In [ ]:
len(val_ds)

4

In [ ]:
for a, b in train_ds:
    print(a.shape)
    print(len(b))
    break

(4, 256, 512, 3)
2


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as KL
 
class SeqConv(KL.Layer):
    
    def __init__(self, y_step, rnn_unit=-1, name='', output_size=-1):
        super(SeqConv, self).__init__(name=name)
        self.y_step = y_step
        self.rnn_unit = rnn_unit
        self.output_size = output_size
    
    def build(self, input_shape):
        _, height, width, channel = input_shape
        if self.rnn_unit < 0:
            self.rnn_unit = channel
        self.rnn_x = KL.GRU(self.rnn_unit, return_sequences=True, name=self.name + '-rnnx', kernel_initializer = 'he_normal')
        self.rnn_x = KL.TimeDistributed(KL.Bidirectional(self.rnn_x), name=self.name + '-bidrnnx')
        
        #self.input_depth = channel
        a = self.y_step
        self.downx = KL.Conv2D(self.rnn_unit, (2*a+1, 2*a+1), strides=(a+1,1), padding='same', name=self.name +'-downx', kernel_initializer = 'he_normal')
        self.upx = KL.UpSampling2D((a+1,1), interpolation='bilinear', name=self.name+'-upx')
        if self.output_size < 0:
            self.output_size = channel
        self.up = KL.Conv2D(self.output_size, (1,1), name=self.name + '-up', kernel_initializer = 'he_normal')
                               
    def call(self, input_tensor):
        x = self.downx(input_tensor)
        x = self.rnn_x(x)
        x = self.upx(x)
        z = tf.concat([input_tensor, x], -1)
        z = self.up(z)
        return z
        
    def get_config(self):
        config = super(SeqConv, self).get_config()
        config.update({"rnn_x": self.rnn_x, "downx":self.downx, "upx":self.upx,\
                       "rnn_unit":self.rnn_unit, "y_step":self.y_step, "output_size":self.output_size,\
                      "up":self.up})
        return config
 
def identity_block(input_tensor, kernel_size, filters, stage, block,
                   use_bias=True, train_bn=True):
    """The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensorimport keras.layers as KL
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        use_bias: Boolean. To use or not use a bias in conv layers.
        train_bn: Boolean. Train or freeze Batch Norm layers
    """
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
 
    x = KL.Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a',
                  use_bias=use_bias, kernel_initializer = 'he_normal')(input_tensor)
    x = KL.BatchNormalization(name=bn_name_base + '2a')(x, training=train_bn)
    x = KL.Activation('relu')(x)
 
    x = KL.Conv2D(nb_filter2, (kernel_size, kernel_size), padding='same',
                  name=conv_name_base + '2b', use_bias=use_bias, kernel_initializer = 'he_normal')(x)
    x = KL.BatchNormalization(name=bn_name_base + '2b')(x, training=train_bn)
    x = KL.Activation('relu')(x)
 
    x = KL.Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c',
                  use_bias=use_bias, kernel_initializer = 'he_normal')(x)
    x = KL.BatchNormalization(name=bn_name_base + '2c')(x, training=train_bn)
 
    x = KL.Add()([x, input_tensor])
    x = KL.Activation('relu', name='res' + str(stage) + block + '_out')(x)
    return x
 
def conv_block(input_tensor, kernel_size, filters, stage, block,
               strides=(2, 2), use_bias=True, train_bn=True):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        use_bias: Boolean. To use or not use a bias in conv layers.
        train_bn: Boolean. Train or freeze Batch Norm layers
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    """
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
 
    x = KL.Conv2D(nb_filter1, (1, 1), strides=strides,
                  name=conv_name_base + '2a', use_bias=use_bias)(input_tensor)
    x = KL.BatchNormalization(name=bn_name_base + '2a')(x, training=train_bn)
    x = KL.Activation('relu')(x)
 
    x = KL.Conv2D(nb_filter2, (kernel_size, kernel_size), padding='same',
                  name=conv_name_base + '2b', use_bias=use_bias)(x)
    x = KL.BatchNormalization(name=bn_name_base + '2b')(x, training=train_bn)
    x = KL.Activation('relu')(x)
 
    x = KL.Conv2D(nb_filter3, (1, 1), name=conv_name_base +
                  '2c', use_bias=use_bias)(x)
    x = KL.BatchNormalization(name=bn_name_base + '2c')(x, training=train_bn)
 
    shortcut = KL.Conv2D(nb_filter3, (1, 1), strides=strides,
                         name=conv_name_base + '1', use_bias=use_bias)(input_tensor)
    shortcut = KL.BatchNormalization(name=bn_name_base + '1')(shortcut, training=train_bn)
 
    x = KL.Add()([x, shortcut])
    x = KL.Activation('relu', name='res' + str(stage) + block + '_out')(x)
    return x
 
 
def stage_shallow(input_tensor, filter_n, depth, stage, train_bn, insert_seqconv=False, rnn_unit=-1):
    x = KL.Conv2D(filter_n, (3,3), padding='same', name=stage+'-conv01', kernel_initializer = 'he_normal')(input_tensor)
    x = KL.BatchNormalization(name=stage + '-bn1')(x, training=train_bn)
    if insert_seqconv:
        x = SeqConv(y_step=1, rnn_unit=rnn_unit, output_size=-1, name=stage+'-seqconv-')(x)
        x = KL.BatchNormalization(name=stage + '-bnseqconv')(x, training=train_bn)
    x = KL.Conv2D(filter_n, (3,3), padding='same', name=stage+'conv11', kernel_initializer = 'he_normal')(x)
    x = KL.BatchNormalization(name=stage + '-bn2')(x, training=train_bn)
    return x
 
def stage_deep(input_tensor, filter_n, depth, stage, train_bn, insert_seqconv=False, rnn_unit=-1):
    filters = [filter_n//2, filter_n//2, filter_n]
    if insert_seqconv:
        x = SeqConv(y_step=1, rnn_unit=rnn_unit, output_size=filter_n, name=stage + '-seqconv-')(input_tensor)
        x = KL.BatchNormalization(name=stage + '-bn')(x, training=train_bn)
    else:
        x = conv_block(input_tensor, 3, filters, stage=stage, block='conv', strides=(1,1), train_bn=train_bn)
    for i in range(depth):
        x = identity_block(x, 3, filters, stage=stage, block='i' + str(i), train_bn=train_bn)
    return x
 
def head_graph(features, filters, name):
    x = features
    for i, filter_ in enumerate(filters):
        x = KL.Conv2D(filter_, (1, 1), padding="SAME", name=name+str(i))(x)
        x = KL.BatchNormalization(name=name+'bn'+str(i))(x)
    return x
 
 
def unet(input_image, name='', train_bn=False): #256x512
    depth = None
    
    C1 = stage_shallow(input_image, 64, depth, name+'C1', train_bn, insert_seqconv=False)
    C2 = KL.MaxPooling2D(pool_size=(2, 2))(C1) #128x256 (/2)
    C2 = stage_shallow(C2, 128, depth, name+'C2', train_bn, insert_seqconv=False)
    C3 = KL.MaxPooling2D(pool_size=(2, 2))(C2) #64x128 (/4)
    C3 = stage_deep(C3, 256, 5, name+'C3', train_bn, insert_seqconv=False)
    C4 = KL.MaxPooling2D(pool_size=(2, 2))(C3) #32x64 (/8)
    C4 = stage_deep(C4, 256, 10, name+'C4', train_bn, insert_seqconv=True, rnn_unit=32)
    C5 = KL.MaxPooling2D(pool_size=(2, 2))(C4) #16x32 (/16)
    C5 = stage_deep(C5, 512, 20, name+'C5', train_bn, insert_seqconv=True, rnn_unit=128)
    C6 = KL.MaxPooling2D(pool_size=(2, 2))(C5) #8x16 (/32)
    
    P6 = stage_shallow(C6, 512, depth, name+'C6P6', train_bn, insert_seqconv=False) #8x16
    P5 = KL.Concatenate(name=name+'concatP6C5')([KL.UpSampling2D(size=(2, 2))(P6), C5]) #16x32
    P5 = stage_shallow(P5, 512, 20, name+'P5', train_bn, insert_seqconv=True, rnn_unit=128)
 
    P4 = KL.Concatenate(name=name+'concatP5C4')([KL.UpSampling2D(size=(2, 2))(P5), C4]) #32x64
    P4 = stage_shallow(P4, 512, 10, name+'P4', train_bn, insert_seqconv=True, rnn_unit=32)
 
    P3 = KL.Concatenate(name=name+'concatP4C3')([KL.UpSampling2D(size=(2, 2))(P4), C3]) #64x128
    P3 = stage_shallow(P3, 256, 5, name+'P3', train_bn, insert_seqconv=False)
 
    P2 = KL.Concatenate(name=name+'concatP3C2')([KL.UpSampling2D(size=(2, 2))(P3), C2]) #256
    P2 = stage_shallow(P2, 256, depth, name+'P2', train_bn, insert_seqconv=False)
    
    P1 = KL.Concatenate(name=name+'concatP2C1')([KL.UpSampling2D(size=(2, 2))(P2), C1]) #512
    P1 = stage_shallow(P1, 128, depth, name+'P1', train_bn, insert_seqconv=False)
    
    return P1

In [ ]:
from tensorflow.keras import Input, Model
 
tf.keras.backend.clear_session()
 
input_size=(256,512)
image = Input(input_size + (3,))
features1 = unet(image, name='USC-', train_bn=True)
premask_output_1 = head_graph(features1, filters=[32,3], name='premask_branch_1')
labels_output_1 = head_graph(features1, filters=[64,37], name='labels_branch_1')
 
model = Model([image], outputs=[premask_output_1, labels_output_1])
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4), loss=[tf.keras.losses.CategoricalCrossentropy(from_logits=True),\
                                                                     tf.keras.losses.BinaryCrossentropy(from_logits=True)])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 512, 3) 0                                            
__________________________________________________________________________________________________
USC-C1-conv01 (Conv2D)          (None, 256, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
USC-C1-bn1 (BatchNormalization) (None, 256, 512, 64) 256         USC-C1-conv01[0][0]              
__________________________________________________________________________________________________
USC-C1conv11 (Conv2D)           (None, 256, 512, 64) 36928       USC-C1-bn1[0][0]                 
_______________________________________________________________________________________

In [ ]:
# model.load_weights('/content/gdrive/My Drive/workspace/dental/checkpoints/model.16.h5')
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/My Drive/workspace/dental/checkpoints2/model.{epoch:02d}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='/content/gdrive/My Drive/workspace/dental/tb2/'),
]
 
history = model.fit(train_ds, epochs=100,initial_epoch=0, validation_data=val_ds, callbacks=my_callbacks, verbose=1)

Epoch 1/100
      1/Unknown - 0s 667us/step - loss: 2.2023 - premask_branch_1bn1_loss: 1.4092 - labels_branch_1bn1_loss: 0.7930WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
     31/Unknown - 255s 8s/step - loss: 1.5939 - premask_branch_1bn1_loss: 0.8059 - labels_branch_1bn1_loss: 0.7880Exception!!!
     49/Unknown - 407s 8s/step - loss: 1.5507 - premask_branch_1bn1_loss: 0.7676 - labels_branch_1bn1_loss: 0.7830Exception!!!
     66/Unknown - 542s 8s/step - loss: 1.5224 - premask_branch_1bn1_loss: 0.7440 - labels_branch_1bn1_loss: 0.7784Exception!!!
     79/Unknown - 642s 8s/step - loss: 1.5063 - premask_branch_1bn1_loss: 0.7313 - labels_branch_1bn1_loss: 0.7751Exception!!!
     92/Unknown - 747s 8s/step - loss: 1.4945 - premask_branch_1bn1_loss: 0.7224 - labe

In [ ]:
!nvidia-smi

Fri Aug 14 10:06:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    50W / 250W |  15561MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda_10.2.89_440.33.01_linux.run

In [ ]:
# for i in range(len(teeth_labels)):
#     cv2.imwrite(os.path.join(temp_dir, '%s.png'% teeth_labels[i]), (masks[:,:,i]/2 + img_croped/2).astype(int))
# for j in range(len(ext_labels)):
#     cv2.imwrite(os.path.join(temp_dir, '%s.png'% ext_labels[j]), (masks[:,:,len(teeth_labels) + j]/2 + img_croped/2).astype(int))